# Anexo 5 - Evidencia de entrenamiento de diversos modelos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.varmax import VARMAX
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/03_casos/df_53_cleaned.csv')

In [3]:
df.head()

,ano,semana,ira_no_neumonia,neumonias_men5,neumonias_60mas,hospitalizados_men5,hospitalizados_60mas,defunciones_men5,defunciones_60mas,sub_reg_nt,...,defunciones_60mas_lag_3_semana,defunciones_60mas_lag_4_semana,defunciones_60mas_lag_5_semana,defunciones_60mas_lag_6_semana,mes,bimestre,trimestre,cuatrimestre,semestre,pandemia_covid
0,2000,1,2660,14,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
1,2000,2,2567,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
2,2000,3,2230,15,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
3,2000,4,2067,12,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,1,1,1,1,1,0
4,2000,5,1677,8,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,2,1,1,1,1,0


In [4]:
df.columns

Index(['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
       'hospitalizados_men5', 'hospitalizados_60mas', 'defunciones_men5',
       'defunciones_60mas', 'sub_reg_nt', 'ira_no_neumonia_lag_1_semana',
       'ira_no_neumonia_lag_2_semana', 'ira_no_neumonia_lag_3_semana',
       'ira_no_neumonia_lag_4_semana', 'ira_no_neumonia_lag_5_semana',
       'ira_no_neumonia_lag_6_semana', 'neumonias_men5_lag_1_semana',
       'neumonias_men5_lag_2_semana', 'neumonias_men5_lag_3_semana',
       'neumonias_men5_lag_4_semana', 'neumonias_men5_lag_5_semana',
       'neumonias_men5_lag_6_semana', 'neumonias_60mas_lag_1_semana',
       'neumonias_60mas_lag_2_semana', 'neumonias_60mas_lag_3_semana',
       'neumonias_60mas_lag_4_semana', 'neumonias_60mas_lag_5_semana',
       'neumonias_60mas_lag_6_semana', 'hospitalizados_men5_lag_1_semana',
       'hospitalizados_men5_lag_2_semana', 'hospitalizados_men5_lag_3_semana',
       'hospitalizados_men5_lag_4_semana', 'hospitalizad

Vamos a entrenar modelos de forecasting para la predicción de `hospitalizados_men5`, considerando los siguientes escenarios:

### Por años [2000 a 2023]

- Entrenar usando:  
  **Variables:** `ano`, `semana`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `pandemia_covid`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   
  **Target:** `hospitalizados_men5`


### Por años [2022 a 2023]

- Entrenar usando:  
  **Variables:** `ano`, `semana`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `ira_no_neumonia`, `neumonias_men5`, `neumonias_60mas`, `hospitalizados_60mas`, `defunciones_men5`, `defunciones_60mas`, y sus variables *lag* (`_lag_1_semana` a `_lag_6_semana`)  
  **Target:** `hospitalizados_men5`

- Entrenar usando:  
  **Variables:** `ano`, `semana`, `hospitalizados_60mas_lag_1_semana`, `defunciones_men5_lag_1_semana`, `defunciones_60mas_lag_1_semana`, `defunciones_men5_lag_3_semana`   
  **Target:** `hospitalizados_men5`

## Modelos: SARIMAX, CatBoost y VARMAX

Entrenaremos tres tipos de modelos para comparar su desempeño:
- **SARIMAX**: Seasonal AutoRegressive Integrated Moving Average with eXogenous variables
- **CatBoost**: Gradient Boosting especializado en datos categóricos y tabulares
- **VARMAX**: Vector AutoRegressive Moving Average with eXogenous variables

In [5]:
# Diccionario para almacenar resultados de todos los modelos
resultados_modelos = {}

In [6]:
# Definir conjuntos de variables según los criterios
variables_sets = {
    'set1_ano_semana': ['ano', 'semana'],
    'set2_con_pandemia': ['ano', 'semana', 'pandemia_covid'],
    'set3_principales': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                         'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    'set4_con_lags': ['ano', 'semana', 'ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
                      'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas'],
    'set5_lags_selectos': ['ano', 'semana', 'hospitalizados_60mas_lag_1_semana',
                           'defunciones_men5_lag_1_semana', 'defunciones_60mas_lag_1_semana',
                           'defunciones_men5_lag_3_semana']
}

# Agregar variables lag al set4
lag_vars = []
for var in ['ira_no_neumonia', 'neumonias_men5', 'neumonias_60mas',
            'hospitalizados_60mas', 'defunciones_men5', 'defunciones_60mas']:
    for i in range(1, 7):
        lag_vars.append(f'{var}_lag_{i}_semana')

variables_sets['set4_con_lags'].extend(lag_vars)

print("Conjuntos de variables definidos:")
for key, vars_list in variables_sets.items():
    print(f"\n{key}: {len(vars_list)} variables")
    print(f"  {', '.join(vars_list[:5])}{'...' if len(vars_list) > 5 else ''}")

Conjuntos de variables definidos:

set1_ano_semana: 2 variables
  ano, semana

set2_con_pandemia: 3 variables
  ano, semana, pandemia_covid

set3_principales: 8 variables
  ano, semana, ira_no_neumonia, neumonias_men5, neumonias_60mas...

set4_con_lags: 44 variables
  ano, semana, ira_no_neumonia, neumonias_men5, neumonias_60mas...

set5_lags_selectos: 6 variables
  ano, semana, hospitalizados_60mas_lag_1_semana, defunciones_men5_lag_1_semana, defunciones_60mas_lag_1_semana...


### Periodo 2000-2023 - Entrenamiento de Modelos

In [7]:
# Preparar datos para periodo 2000-2023
df_2000_2023 = df.copy().sort_values(['ano', 'semana']).reset_index(drop=True)
y_full = df_2000_2023['hospitalizados_men5']
split_idx = int(len(y_full) * 0.8)

print(f"Total observaciones 2000-2023: {len(y_full)}")
print(f"Train: {split_idx} observaciones")
print(f"Test: {len(y_full) - split_idx} observaciones")

Total observaciones 2000-2023: 1248
Train: 998 observaciones
Test: 250 observaciones


In [8]:
# Entrenar modelos para periodo 2000-2023
sets_2000_2023 = ['set1_ano_semana', 'set2_con_pandemia', 'set3_principales', 'set4_con_lags', 'set5_lags_selectos']

for set_name in sets_2000_2023:
    exog_vars = variables_sets[set_name]
    X = df_2000_2023[exog_vars]
    y = df_2000_2023['hospitalizados_men5']
    
    X_train = X[:split_idx]
    X_test = X[split_idx:]
    y_train = y[:split_idx]
    y_test = y[split_idx:]
    
    # Nombre descriptivo
    if set_name == 'set1_ano_semana':
        var_desc = 'ano, semana'
    elif set_name == 'set2_con_pandemia':
        var_desc = 'ano, semana, pandemia_covid'
    elif set_name == 'set3_principales':
        var_desc = 'principales'
    elif set_name == 'set4_con_lags':
        var_desc = 'principales + lags'
    else:
        var_desc = 'lags selectos'
    
    # ==================== SARIMAX ====================
    try:
        model_sarimax = SARIMAX(y_train, exog=X_train, order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 52),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
        fit_sarimax = model_sarimax.fit(disp=False)
        pred_sarimax = fit_sarimax.forecast(steps=len(y_test), exog=X_test)
        
        resultados_modelos[f'SARIMAX_2000-2023_{set_name}'] = {
            'modelo': 'SARIMAX',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_sarimax),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_sarimax)),
            'R2': r2_score(y_test, pred_sarimax)
        }
        print(f"✓ SARIMAX completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en SARIMAX {set_name}: {e}")
    
    # ==================== CatBoost ====================
    try:
        model_catboost = CatBoostRegressor(
            iterations=500,
            learning_rate=0.05,
            depth=6,
            loss_function='RMSE',
            verbose=False,
            random_state=42
        )
        model_catboost.fit(X_train, y_train)
        pred_catboost = model_catboost.predict(X_test)
        
        resultados_modelos[f'CatBoost_2000-2023_{set_name}'] = {
            'modelo': 'CatBoost',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_catboost),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_catboost)),
            'R2': r2_score(y_test, pred_catboost)
        }
        print(f"✓ CatBoost completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en CatBoost {set_name}: {e}")
    
    # ==================== VARMAX ====================
    # VARMAX requiere múltiples variables endógenas, usaremos hospitalizados_men5 y hospitalizados_60mas
    try:
        # Preparar variables endógenas para VARMAX
        endog_vars = ['hospitalizados_men5', 'hospitalizados_60mas']
        y_varmax = df_2000_2023[endog_vars]
        y_varmax_train = y_varmax[:split_idx]
        y_varmax_test = y_varmax[split_idx:]
        
        model_varmax = VARMAX(y_varmax_train, exog=X_train, order=(1, 1),
                              enforce_stationarity=False,
                              enforce_invertibility=False)
        fit_varmax = model_varmax.fit(disp=False, maxiter=100)
        pred_varmax = fit_varmax.forecast(steps=len(y_test), exog=X_test)
        
        # Extraer predicciones solo para hospitalizados_men5
        pred_varmax_men5 = pred_varmax.iloc[:, 0]
        
        resultados_modelos[f'VARMAX_2000-2023_{set_name}'] = {
            'modelo': 'VARMAX',
            'periodo': '2000-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_varmax_men5),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_varmax_men5)),
            'R2': r2_score(y_test, pred_varmax_men5)
        }
        print(f"✓ VARMAX completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en VARMAX {set_name}: {e}")
    
    print(f"Completado conjunto: {set_name} para periodo 2000-2023\n")

print(f"\n{'='*80}")
print(f"Total de modelos entrenados (2000-2023): {len(resultados_modelos)}")
print(f"{'='*80}")

✓ SARIMAX completado: set1_ano_semana
✓ CatBoost completado: set1_ano_semana
✗ Error en VARMAX set1_ano_semana: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set1_ano_semana para periodo 2000-2023

✓ SARIMAX completado: set2_con_pandemia
✓ CatBoost completado: set2_con_pandemia
✗ Error en VARMAX set2_con_pandemia: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set2_con_pandemia para periodo 2000-2023

✓ SARIMAX completado: set3_principales
✓ CatBoost completado: set3_principales
✗ Error en VARMAX set3_principales: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set3_principales para periodo 2000-2023

✓ SARIMAX completado: set4_con_lags
✓ CatBoost completado: set4_con_lags
✗ Error en VARMAX set4_con_lags: Cannot cast ufunc 'subtract' output from dtyp

### Periodo 2022-2023 - Entrenamiento de Modelos

In [9]:
# Preparar datos para periodo 2022-2023
df_2022_2023 = df[df['ano'] >= 2022].copy().sort_values(['ano', 'semana']).reset_index(drop=True)
y_full_2022 = df_2022_2023['hospitalizados_men5']
split_idx_2022 = int(len(y_full_2022) * 0.8)

print(f"Total observaciones 2022-2023: {len(y_full_2022)}")
print(f"Train: {split_idx_2022} observaciones")
print(f"Test: {len(y_full_2022) - split_idx_2022} observaciones")

Total observaciones 2022-2023: 104
Train: 83 observaciones
Test: 21 observaciones


In [10]:
# Entrenar modelos para periodo 2022-2023
sets_2022_2023 = ['set1_ano_semana', 'set3_principales', 'set4_con_lags', 'set5_lags_selectos']

for set_name in sets_2022_2023:
    exog_vars = variables_sets[set_name]
    X = df_2022_2023[exog_vars]
    y = df_2022_2023['hospitalizados_men5']
    
    X_train = X[:split_idx_2022]
    X_test = X[split_idx_2022:]
    y_train = y[:split_idx_2022]
    y_test = y[split_idx_2022:]
    
    # Nombre descriptivo
    if set_name == 'set1_ano_semana':
        var_desc = 'ano, semana'
    elif set_name == 'set3_principales':
        var_desc = 'principales'
    elif set_name == 'set4_con_lags':
        var_desc = 'principales + lags'
    else:
        var_desc = 'lags selectos'
    
    # ==================== SARIMAX ====================
    try:
        model_sarimax = SARIMAX(y_train, exog=X_train, order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 52),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
        fit_sarimax = model_sarimax.fit(disp=False)
        pred_sarimax = fit_sarimax.forecast(steps=len(y_test), exog=X_test)
        
        resultados_modelos[f'SARIMAX_2022-2023_{set_name}'] = {
            'modelo': 'SARIMAX',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_sarimax),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_sarimax)),
            'R2': r2_score(y_test, pred_sarimax)
        }
        print(f"✓ SARIMAX completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en SARIMAX {set_name}: {e}")
    
    # ==================== CatBoost ====================
    try:
        model_catboost = CatBoostRegressor(
            iterations=500,
            learning_rate=0.05,
            depth=6,
            loss_function='RMSE',
            verbose=False,
            random_state=42
        )
        model_catboost.fit(X_train, y_train)
        pred_catboost = model_catboost.predict(X_test)
        
        resultados_modelos[f'CatBoost_2022-2023_{set_name}'] = {
            'modelo': 'CatBoost',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_catboost),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_catboost)),
            'R2': r2_score(y_test, pred_catboost)
        }
        print(f"✓ CatBoost completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en CatBoost {set_name}: {e}")
    
    # ==================== VARMAX ====================
    try:
        # Preparar variables endógenas para VARMAX
        endog_vars = ['hospitalizados_men5', 'hospitalizados_60mas']
        y_varmax = df_2022_2023[endog_vars]
        y_varmax_train = y_varmax[:split_idx_2022]
        y_varmax_test = y_varmax[split_idx_2022:]
        
        model_varmax = VARMAX(y_varmax_train, exog=X_train, order=(1, 1),
                              enforce_stationarity=False,
                              enforce_invertibility=False)
        fit_varmax = model_varmax.fit(disp=False, maxiter=100)
        pred_varmax = fit_varmax.forecast(steps=len(y_test), exog=X_test)
        
        # Extraer predicciones solo para hospitalizados_men5
        pred_varmax_men5 = pred_varmax.iloc[:, 0]
        
        resultados_modelos[f'VARMAX_2022-2023_{set_name}'] = {
            'modelo': 'VARMAX',
            'periodo': '2022-2023',
            'variables': var_desc,
            'MAE': mean_absolute_error(y_test, pred_varmax_men5),
            'RMSE': np.sqrt(mean_squared_error(y_test, pred_varmax_men5)),
            'R2': r2_score(y_test, pred_varmax_men5)
        }
        print(f"✓ VARMAX completado: {set_name}")
    except Exception as e:
        print(f"✗ Error en VARMAX {set_name}: {e}")
    
    print(f"Completado conjunto: {set_name} para periodo 2022-2023\n")

print(f"\n{'='*80}")
print(f"Total de modelos entrenados (todos): {len(resultados_modelos)}")
print(f"{'='*80}")

✓ SARIMAX completado: set1_ano_semana
✓ CatBoost completado: set1_ano_semana
✗ Error en VARMAX set1_ano_semana: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set1_ano_semana para periodo 2022-2023

✓ SARIMAX completado: set3_principales
✓ CatBoost completado: set3_principales
✗ Error en VARMAX set3_principales: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set3_principales para periodo 2022-2023

✓ SARIMAX completado: set4_con_lags
✓ CatBoost completado: set4_con_lags
✗ Error en VARMAX set4_con_lags: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'
Completado conjunto: set4_con_lags para periodo 2022-2023

✓ SARIMAX completado: set5_lags_selectos
✓ CatBoost completado: set5_lags_selectos
✗ Error en VARMAX set5_lags_selectos: Cannot cast ufunc 'subtract' output from dtype

---

## Tabla Comparativa Completa - Formato Pivot Table

In [11]:
# Crear DataFrame con todos los resultados
df_resultados = pd.DataFrame(resultados_modelos).T
df_resultados = df_resultados.reset_index()
df_resultados.rename(columns={'index': 'nombre_completo'}, inplace=True)

# Redondear métricas
df_resultados[['MAE', 'RMSE', 'R2']] = df_resultados[['MAE', 'RMSE', 'R2']].round(4)

# Crear tabla pivote
df_pivot = df_resultados[['periodo', 'modelo', 'variables', 'MAE', 'RMSE', 'R2']].copy()
df_pivot = df_pivot.sort_values(['periodo', 'modelo', 'variables']).reset_index(drop=True)

print("="*100)
print("TABLA COMPARATIVA - MODELOS SARIMAX, CatBoost y VARMAX")
print("="*100)
print("\nTarget: hospitalizados_men5")
print("\nColumnas:")
print("  - Periodo: Rango de años utilizado")
print("  - Modelo: SARIMAX, CatBoost o VARMAX")
print("  - Variables: Conjunto de variables utilizadas")
print("  - MAE: Mean Absolute Error (menor es mejor)")
print("  - RMSE: Root Mean Squared Error (menor es mejor)")
print("  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)")
print("\n" + "="*100)
print(df_pivot.to_string(index=True))
print("\n" + "="*100)

TABLA COMPARATIVA - MODELOS SARIMAX, CatBoost y VARMAX

Target: hospitalizados_men5

Columnas:
  - Periodo: Rango de años utilizado
  - Modelo: SARIMAX, CatBoost o VARMAX
  - Variables: Conjunto de variables utilizadas
  - MAE: Mean Absolute Error (menor es mejor)
  - RMSE: Root Mean Squared Error (menor es mejor)
  - R²: Coeficiente de Determinación (mayor es mejor, cercano a 1)

      periodo    modelo                    variables         MAE         RMSE             R2
0   2000-2023  CatBoost                  ano, semana    3.695857     4.450672      -0.344135
1   2000-2023  CatBoost  ano, semana, pandemia_covid    3.658615     4.360237      -0.290065
2   2000-2023  CatBoost                lags selectos    3.250005     3.972293      -0.070716
3   2000-2023  CatBoost                  principales    2.785486      3.33348       0.245972
4   2000-2023  CatBoost           principales + lags    2.648623     3.288715       0.266088
5   2000-2023   SARIMAX                  ano, semana    3.

### Análisis de Mejores Modelos

In [12]:
print("="*100)
print("ANÁLISIS DE MEJORES MODELOS")
print("="*100)

# Mejor modelo global
print("\n1. MEJOR MODELO GLOBAL:")
print("-" * 100)
mejor_mae = df_pivot.loc[df_pivot['MAE'].idxmin()]
mejor_rmse = df_pivot.loc[df_pivot['RMSE'].idxmin()]
mejor_r2 = df_pivot.loc[df_pivot['R2'].idxmax()]

print(f"\nMenor MAE: {mejor_mae['MAE']:.4f}")
print(f"  Modelo: {mejor_mae['modelo']}")
print(f"  Periodo: {mejor_mae['periodo']}")
print(f"  Variables: {mejor_mae['variables']}")

print(f"\nMenor RMSE: {mejor_rmse['RMSE']:.4f}")
print(f"  Modelo: {mejor_rmse['modelo']}")
print(f"  Periodo: {mejor_rmse['periodo']}")
print(f"  Variables: {mejor_rmse['variables']}")

print(f"\nMayor R²: {mejor_r2['R2']:.4f}")
print(f"  Modelo: {mejor_r2['modelo']}")
print(f"  Periodo: {mejor_r2['periodo']}")
print(f"  Variables: {mejor_r2['variables']}")

# Mejor modelo por periodo
print("\n" + "="*100)
print("2. MEJOR MODELO POR PERIODO:")
print("-" * 100)
for periodo in df_pivot['periodo'].unique():
    df_periodo = df_pivot[df_pivot['periodo'] == periodo]
    mejor = df_periodo.loc[df_periodo['R2'].idxmax()]
    
    print(f"\n{periodo}:")
    print(f"  Modelo: {mejor['modelo']}")
    print(f"  Variables: {mejor['variables']}")
    print(f"  MAE: {mejor['MAE']:.4f}")
    print(f"  RMSE: {mejor['RMSE']:.4f}")
    print(f"  R²: {mejor['R2']:.4f}")

# Mejor modelo por tipo
print("\n" + "="*100)
print("3. MEJOR CONFIGURACIÓN POR TIPO DE MODELO:")
print("-" * 100)
for modelo_tipo in df_pivot['modelo'].unique():
    df_modelo = df_pivot[df_pivot['modelo'] == modelo_tipo]
    mejor = df_modelo.loc[df_modelo['R2'].idxmax()]
    
    print(f"\n{modelo_tipo}:")
    print(f"  Periodo: {mejor['periodo']}")
    print(f"  Variables: {mejor['variables']}")
    print(f"  MAE: {mejor['MAE']:.4f}")
    print(f"  RMSE: {mejor['RMSE']:.4f}")
    print(f"  R²: {mejor['R2']:.4f}")

# Desempeño promedio por conjunto de variables
print("\n" + "="*100)
print("4. DESEMPEÑO PROMEDIO POR CONJUNTO DE VARIABLES:")
print("-" * 100)
agg_vars = df_pivot.groupby('variables')[['MAE', 'RMSE', 'R2']].mean().round(4)
agg_vars = agg_vars.sort_values('R2', ascending=False)
print(agg_vars)

print("\n" + "="*100)

ANÁLISIS DE MEJORES MODELOS

1. MEJOR MODELO GLOBAL:
----------------------------------------------------------------------------------------------------

Menor MAE: 1.8704
  Modelo: CatBoost
  Periodo: 2022-2023
  Variables: ano, semana

Menor RMSE: 2.1623
  Modelo: CatBoost
  Periodo: 2022-2023
  Variables: ano, semana

Mayor R²: 0.2661
  Modelo: CatBoost
  Periodo: 2000-2023
  Variables: principales + lags

2. MEJOR MODELO POR PERIODO:
----------------------------------------------------------------------------------------------------

2000-2023:
  Modelo: CatBoost
  Variables: principales + lags
  MAE: 2.6486
  RMSE: 3.2887
  R²: 0.2661

2022-2023:
  Modelo: CatBoost
  Variables: ano, semana
  MAE: 1.8704
  RMSE: 2.1623
  R²: -0.1532

3. MEJOR CONFIGURACIÓN POR TIPO DE MODELO:
----------------------------------------------------------------------------------------------------

CatBoost:
  Periodo: 2000-2023
  Variables: principales + lags
  MAE: 2.6486
  RMSE: 3.2887
  R²: 0.2661



### Visualización de Resultados

### Exportar Resultados

In [13]:
# Guardar resultados en CSV
output_path = '../data/resultados_modelos_sarimax_catboost_varmax.csv'
df_pivot.to_csv(output_path, index=False)
print(f"Resultados guardados en: {output_path}")

# Resumen final
print("\n" + "="*100)
print("RESUMEN FINAL")
print("="*100)
print(f"\nTotal de modelos entrenados: {len(df_pivot)}")
print(f"Periodos analizados: {df_pivot['periodo'].nunique()}")
print(f"Tipos de modelos: {', '.join(df_pivot['modelo'].unique())}")
print(f"Conjuntos de variables: {df_pivot['variables'].nunique()}")
print("\nModelos:")
print("  - SARIMAX: Seasonal ARIMA with eXogenous variables")
print("  - CatBoost: Gradient Boosting optimizado")
print("  - VARMAX: Vector AutoRegressive Moving Average with eXogenous")
print("\nArchivo de resultados generado exitosamente.")
print("="*100)

Resultados guardados en: ../data/resultados_modelos_sarimax_catboost_varmax.csv

RESUMEN FINAL

Total de modelos entrenados: 18
Periodos analizados: 2
Tipos de modelos: CatBoost, SARIMAX
Conjuntos de variables: 5

Modelos:
  - SARIMAX: Seasonal ARIMA with eXogenous variables
  - CatBoost: Gradient Boosting optimizado
  - VARMAX: Vector AutoRegressive Moving Average with eXogenous

Archivo de resultados generado exitosamente.
